In [1]:
####DATA LOADING######
import pandas as pd
test = pd.read_csv('/mnt/data/Speech-Recognition-system_wav2vec-20/test.csv')
#train = pd.read_csv('/mnt/data/Speech-Recognition-system_wav2vec-20/df_text.csv')
#df_segments = pd.DataFrame(df_segments)
#df_text = pd.DataFrame(df_text)

In [2]:
#%%capture
#!pip install -U transformers
#!pip install datasets
#!pip install fsspec==2021.5.0
#!pip install jiwer==2.2.0

In [3]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [4]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

In [5]:
transformers.logging.get_verbosity()

0

In [6]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [4]:
test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
test['chunk_ID'] = test['chunk_ID'] + '.wav'

In [9]:
test['audio_file_ID'] = test['audio_file_ID'] + '.wav'

In [10]:
test

,chunk_ID,audio_file_ID,start_time,end_time,sentence
0,103085_w5Jyq3XMbb3WwiKQ_0000.wav,w5Jyq3XMbb3WwiKQ.wav,0.0,8.00,लिबर ऑफिस impress में एक प्रस्तुति document बन...
1,103085_w5Jyq3XMbb3WwiKQ_0001.wav,w5Jyq3XMbb3WwiKQ.wav,8.0,21.00,इस tutorial में हम impress window के भागों के ...
2,103085_w5Jyq3XMbb3WwiKQ_0002.wav,w5Jyq3XMbb3WwiKQ.wav,21.0,29.00,यहाँ हम अपने ऑपरेटिंग सिस्टम के रूप में gnu/li...
3,103085_w5Jyq3XMbb3WwiKQ_0003.wav,w5Jyq3XMbb3WwiKQ.wav,29.0,35.00,चलिए अपनी प्रस्तुति प्रेजैटेशन sample impress ...
4,103085_w5Jyq3XMbb3WwiKQ_0004.wav,w5Jyq3XMbb3WwiKQ.wav,35.0,39.00,चलिए देखते हैं कि screen पर क्या क्या है
...,...,...,...,...,...
3131,957491_4xyIm2P6Xzlin341_0166.wav,4xyIm2P6Xzlin341.wav,866.0,872.00,अधिक जानकारी के लिए कृपया contact @spokentutor...
3132,957491_4xyIm2P6Xzlin341_0167.wav,4xyIm2P6Xzlin341.wav,872.0,876.00,spoken tutorial project talktoa teacher projec...
3133,957491_4xyIm2P6Xzlin341_0168.wav,4xyIm2P6Xzlin341.wav,876.0,884.00,यह भारत सरकार के एमएचआरडी के आईसीटी के माध्यम ...
3134,957491_4xyIm2P6Xzlin341_0169.wav,4xyIm2P6Xzlin341.wav,884.0,895.00,इस mission पर अधिक जानकारी दिए गए लिंक पर उपलब...


In [1]:
import pandas as pd
text_train = pd.read_csv('/mnt/data/ASR_Dataset/Hindi-English/train/transcripts/text') 

In [9]:
segments_train = pd.read_csv('/mnt/data/ASR_Dataset/Hindi-English/train/transcripts/segments') 

In [10]:
segments_train

,100051_GigsEKRBMhvNwolC_0000 GigsEKRBMhvNwolC 0.0 9.0
0,100051_GigsEKRBMhvNwolC_0001 GigsEKRBMhvNwolC ...
1,100051_GigsEKRBMhvNwolC_0002 GigsEKRBMhvNwolC ...
2,100051_GigsEKRBMhvNwolC_0003 GigsEKRBMhvNwolC ...
3,100051_GigsEKRBMhvNwolC_0004 GigsEKRBMhvNwolC ...
4,100051_GigsEKRBMhvNwolC_0005 GigsEKRBMhvNwolC ...
...,...
52819,996368_n4ACC4gnqW0IM1wW_0081 n4ACC4gnqW0IM1wW ...
52820,996368_n4ACC4gnqW0IM1wW_0082 n4ACC4gnqW0IM1wW ...
52821,996368_n4ACC4gnqW0IM1wW_0083 n4ACC4gnqW0IM1wW ...
52822,996368_n4ACC4gnqW0IM1wW_0084 n4ACC4gnqW0IM1wW ...


In [12]:
segments_train.columns

Index(['100051_GigsEKRBMhvNwolC_0000 GigsEKRBMhvNwolC 0.0 9.0'], dtype='object')

In [13]:
segments_train.rename(columns={'100051_GigsEKRBMhvNwolC_0000 GigsEKRBMhvNwolC 0.0 9.0': 'chunk_ID'}, inplace = True)
segments_train

,chunk_ID
0,100051_GigsEKRBMhvNwolC_0001 GigsEKRBMhvNwolC ...
1,100051_GigsEKRBMhvNwolC_0002 GigsEKRBMhvNwolC ...
2,100051_GigsEKRBMhvNwolC_0003 GigsEKRBMhvNwolC ...
3,100051_GigsEKRBMhvNwolC_0004 GigsEKRBMhvNwolC ...
4,100051_GigsEKRBMhvNwolC_0005 GigsEKRBMhvNwolC ...
...,...
52819,996368_n4ACC4gnqW0IM1wW_0081 n4ACC4gnqW0IM1wW ...
52820,996368_n4ACC4gnqW0IM1wW_0082 n4ACC4gnqW0IM1wW ...
52821,996368_n4ACC4gnqW0IM1wW_0083 n4ACC4gnqW0IM1wW ...
52822,996368_n4ACC4gnqW0IM1wW_0084 n4ACC4gnqW0IM1wW ...


In [14]:
import pandas as pd
segments_train[['chunk_ID', 'audio_file_ID', 'start_time', 'end_time']] = segments_train.chunk_ID.str.split(" ", expand=True,)

In [15]:
segments_train

,chunk_ID,audio_file_ID,start_time,end_time
0,100051_GigsEKRBMhvNwolC_0001,GigsEKRBMhvNwolC,9.0,12.0
1,100051_GigsEKRBMhvNwolC_0002,GigsEKRBMhvNwolC,12.0,14.0
2,100051_GigsEKRBMhvNwolC_0003,GigsEKRBMhvNwolC,14.0,17.0
3,100051_GigsEKRBMhvNwolC_0004,GigsEKRBMhvNwolC,17.0,22.0
4,100051_GigsEKRBMhvNwolC_0005,GigsEKRBMhvNwolC,22.0,28.0
...,...,...,...,...
52819,996368_n4ACC4gnqW0IM1wW_0081,n4ACC4gnqW0IM1wW,599.0,605.0
52820,996368_n4ACC4gnqW0IM1wW_0082,n4ACC4gnqW0IM1wW,605.0,613.0
52821,996368_n4ACC4gnqW0IM1wW_0083,n4ACC4gnqW0IM1wW,613.0,618.0
52822,996368_n4ACC4gnqW0IM1wW_0084,n4ACC4gnqW0IM1wW,618.0,629.0


In [16]:
top_row = pd.DataFrame({'chunk_ID':['100051_GigsEKRBMhvNwolC_0000'], 'audio_file_ID':['GigsEKRBMhvNwolC'], 'start_time':[0.0], 'end_time':[9.0]})
segments_train = pd.concat([top_row, segments_train]).reset_index(drop=True)
segments_train

,chunk_ID,audio_file_ID,start_time,end_time
0,100051_GigsEKRBMhvNwolC_0000,GigsEKRBMhvNwolC,0.0,9.0
1,100051_GigsEKRBMhvNwolC_0001,GigsEKRBMhvNwolC,9.0,12.0
2,100051_GigsEKRBMhvNwolC_0002,GigsEKRBMhvNwolC,12.0,14.0
3,100051_GigsEKRBMhvNwolC_0003,GigsEKRBMhvNwolC,14.0,17.0
4,100051_GigsEKRBMhvNwolC_0004,GigsEKRBMhvNwolC,17.0,22.0
...,...,...,...,...
52820,996368_n4ACC4gnqW0IM1wW_0081,n4ACC4gnqW0IM1wW,599.0,605.0
52821,996368_n4ACC4gnqW0IM1wW_0082,n4ACC4gnqW0IM1wW,605.0,613.0
52822,996368_n4ACC4gnqW0IM1wW_0083,n4ACC4gnqW0IM1wW,613.0,618.0
52823,996368_n4ACC4gnqW0IM1wW_0084,n4ACC4gnqW0IM1wW,618.0,629.0


In [3]:
text_train.columns

Index(['100051_GigsEKRBMhvNwolC_0000 दोस्तों bash में nested और multilevel if statement के spoken tutorial में आपका स्वागत है'], dtype='object')

In [4]:
text_train.rename(columns={'100051_GigsEKRBMhvNwolC_0000 दोस्तों bash में nested और multilevel if statement के spoken tutorial में आपका स्वागत है': 'sentence'}, inplace = True)
text_train

,sentence
0,100051_GigsEKRBMhvNwolC_0001 इस tutorial में ह...
1,100051_GigsEKRBMhvNwolC_0002 nested ifelse और
2,100051_GigsEKRBMhvNwolC_0003 multilevel ifelse...
3,100051_GigsEKRBMhvNwolC_0004 हम यह कुछ उदाहरण ...
4,100051_GigsEKRBMhvNwolC_0005 इस tutorial के अन...
...,...
52819,996368_n4ACC4gnqW0IM1wW_0081 निम्नलिखित link प...
52820,996368_n4ACC4gnqW0IM1wW_0082 spoken tutorial p...
52821,996368_n4ACC4gnqW0IM1wW_0083 कृपया मिनट और सेक...
52822,996368_n4ACC4gnqW0IM1wW_0084 spoken tutorial p...


In [5]:
new = text_train['sentence'].str.split(" ", n= 1, expand = True)
new

,0,1
0,100051_GigsEKRBMhvNwolC_0001,इस tutorial में हम निम्न के बारे में सीखेंगे
1,100051_GigsEKRBMhvNwolC_0002,nested ifelse और
2,100051_GigsEKRBMhvNwolC_0003,multilevel ifelse statement
3,100051_GigsEKRBMhvNwolC_0004,हम यह कुछ उदाहरण उपयोग करके करेंगे
4,100051_GigsEKRBMhvNwolC_0005,इस tutorial के अनुसरण के लिए आपको लिनक्स opera...
...,...,...
52819,996368_n4ACC4gnqW0IM1wW_0081,निम्नलिखित link पर video spoken tutorial proje...
52820,996368_n4ACC4gnqW0IM1wW_0082,spoken tutorial project team कार्यशालाएं आयोजि...
52821,996368_n4ACC4gnqW0IM1wW_0083,कृपया मिनट और सेकंड के साथ अपने प्रश्नों को इस...
52822,996368_n4ACC4gnqW0IM1wW_0084,spoken tutorial project एनएमईआईसीटी एमएचआरडी भ...


In [7]:
text_train['chunk_ID'] = new[0]
text_train['sentence'] = new[1]

In [8]:
text_train

,sentence,chunk_ID
0,इस tutorial में हम निम्न के बारे में सीखेंगे,100051_GigsEKRBMhvNwolC_0001
1,nested ifelse और,100051_GigsEKRBMhvNwolC_0002
2,multilevel ifelse statement,100051_GigsEKRBMhvNwolC_0003
3,हम यह कुछ उदाहरण उपयोग करके करेंगे,100051_GigsEKRBMhvNwolC_0004
4,इस tutorial के अनुसरण के लिए आपको लिनक्स opera...,100051_GigsEKRBMhvNwolC_0005
...,...,...
52819,निम्नलिखित link पर video spoken tutorial proje...,996368_n4ACC4gnqW0IM1wW_0081
52820,spoken tutorial project team कार्यशालाएं आयोजि...,996368_n4ACC4gnqW0IM1wW_0082
52821,कृपया मिनट और सेकंड के साथ अपने प्रश्नों को इस...,996368_n4ACC4gnqW0IM1wW_0083
52822,spoken tutorial project एनएमईआईसीटी एमएचआरडी भ...,996368_n4ACC4gnqW0IM1wW_0084


In [17]:
top_row = pd.DataFrame({'chunk_ID':['100051_GigsEKRBMhvNwolC_0000'], 'sentence':['दोस्तों bash में nested और multilevel if statement के spoken tutorial में आपका स्वागत है']})
text_train = pd.concat([top_row, text_train]).reset_index(drop=True)
text_train

,chunk_ID,sentence
0,100051_GigsEKRBMhvNwolC_0000,दोस्तों bash में nested और multilevel if state...
1,100051_GigsEKRBMhvNwolC_0001,इस tutorial में हम निम्न के बारे में सीखेंगे
2,100051_GigsEKRBMhvNwolC_0002,nested ifelse और
3,100051_GigsEKRBMhvNwolC_0003,multilevel ifelse statement
4,100051_GigsEKRBMhvNwolC_0004,हम यह कुछ उदाहरण उपयोग करके करेंगे
...,...,...
52820,996368_n4ACC4gnqW0IM1wW_0081,निम्नलिखित link पर video spoken tutorial proje...
52821,996368_n4ACC4gnqW0IM1wW_0082,spoken tutorial project team कार्यशालाएं आयोजि...
52822,996368_n4ACC4gnqW0IM1wW_0083,कृपया मिनट और सेकंड के साथ अपने प्रश्नों को इस...
52823,996368_n4ACC4gnqW0IM1wW_0084,spoken tutorial project एनएमईआईसीटी एमएचआरडी भ...


In [18]:
train = pd.concat([text_train, segments_train], axis = 1)

In [19]:
train

,chunk_ID,sentence,chunk_ID,audio_file_ID,start_time,end_time
0,100051_GigsEKRBMhvNwolC_0000,दोस्तों bash में nested और multilevel if state...,100051_GigsEKRBMhvNwolC_0000,GigsEKRBMhvNwolC,0.0,9.0
1,100051_GigsEKRBMhvNwolC_0001,इस tutorial में हम निम्न के बारे में सीखेंगे,100051_GigsEKRBMhvNwolC_0001,GigsEKRBMhvNwolC,9.0,12.0
2,100051_GigsEKRBMhvNwolC_0002,nested ifelse और,100051_GigsEKRBMhvNwolC_0002,GigsEKRBMhvNwolC,12.0,14.0
3,100051_GigsEKRBMhvNwolC_0003,multilevel ifelse statement,100051_GigsEKRBMhvNwolC_0003,GigsEKRBMhvNwolC,14.0,17.0
4,100051_GigsEKRBMhvNwolC_0004,हम यह कुछ उदाहरण उपयोग करके करेंगे,100051_GigsEKRBMhvNwolC_0004,GigsEKRBMhvNwolC,17.0,22.0
...,...,...,...,...,...,...
52820,996368_n4ACC4gnqW0IM1wW_0081,निम्नलिखित link पर video spoken tutorial proje...,996368_n4ACC4gnqW0IM1wW_0081,n4ACC4gnqW0IM1wW,599.0,605.0
52821,996368_n4ACC4gnqW0IM1wW_0082,spoken tutorial project team कार्यशालाएं आयोजि...,996368_n4ACC4gnqW0IM1wW_0082,n4ACC4gnqW0IM1wW,605.0,613.0
52822,996368_n4ACC4gnqW0IM1wW_0083,कृपया मिनट और सेकंड के साथ अपने प्रश्नों को इस...,996368_n4ACC4gnqW0IM1wW_0083,n4ACC4gnqW0IM1wW,613.0,618.0
52823,996368_n4ACC4gnqW0IM1wW_0084,spoken tutorial project एनएमईआईसीटी एमएचआरडी भ...,996368_n4ACC4gnqW0IM1wW_0084,n4ACC4gnqW0IM1wW,618.0,629.0


In [20]:
train = train.loc[:, ~train.columns.duplicated()] 

In [21]:
train

,chunk_ID,sentence,audio_file_ID,start_time,end_time
0,100051_GigsEKRBMhvNwolC_0000,दोस्तों bash में nested और multilevel if state...,GigsEKRBMhvNwolC,0.0,9.0
1,100051_GigsEKRBMhvNwolC_0001,इस tutorial में हम निम्न के बारे में सीखेंगे,GigsEKRBMhvNwolC,9.0,12.0
2,100051_GigsEKRBMhvNwolC_0002,nested ifelse और,GigsEKRBMhvNwolC,12.0,14.0
3,100051_GigsEKRBMhvNwolC_0003,multilevel ifelse statement,GigsEKRBMhvNwolC,14.0,17.0
4,100051_GigsEKRBMhvNwolC_0004,हम यह कुछ उदाहरण उपयोग करके करेंगे,GigsEKRBMhvNwolC,17.0,22.0
...,...,...,...,...,...
52820,996368_n4ACC4gnqW0IM1wW_0081,निम्नलिखित link पर video spoken tutorial proje...,n4ACC4gnqW0IM1wW,599.0,605.0
52821,996368_n4ACC4gnqW0IM1wW_0082,spoken tutorial project team कार्यशालाएं आयोजि...,n4ACC4gnqW0IM1wW,605.0,613.0
52822,996368_n4ACC4gnqW0IM1wW_0083,कृपया मिनट और सेकंड के साथ अपने प्रश्नों को इस...,n4ACC4gnqW0IM1wW,613.0,618.0
52823,996368_n4ACC4gnqW0IM1wW_0084,spoken tutorial project एनएमईआईसीटी एमएचआरडी भ...,n4ACC4gnqW0IM1wW,618.0,629.0


In [ ]:
train['chunk_ID'] = train['chunk_ID'] + '.wav'

In [ ]:
train['audio_file_ID'] = train['chunk_ID'] + '.wav'

In [8]:
import pandas as pd
df_segments[['chunk_IDS', 'audio_file_ID', 'start_time', 'end_time']] = df_segments.chunk_ID.str.split(" ", expand=True,)

AttributeError: 'DataFrame' object has no attribute 'chunk_ID'

In [9]:
df_segments.drop(['chunk_ID'],axis='columns', inplace =True)
df_segments

KeyError: "['chunk_ID'] not found in axis"

In [10]:
df_segments.value_counts()


100051_GigsEKRBMhvNwolC_0000 GigsEKRBMhvNwolC 0.0 9.0     
100051_GigsEKRBMhvNwolC_0001 GigsEKRBMhvNwolC 9.0 12.0        1
669313_kHCF9cpAg7G7HLb3_0050 kHCF9cpAg7G7HLb3 312.0 324.0     1
669313_kHCF9cpAg7G7HLb3_0040 kHCF9cpAg7G7HLb3 254.0 261.0     1
669313_kHCF9cpAg7G7HLb3_0041 kHCF9cpAg7G7HLb3 261.0 265.0     1
669313_kHCF9cpAg7G7HLb3_0042 kHCF9cpAg7G7HLb3 265.0 267.0     1
                                                             ..
384371_9OL66l59ysIB2YCS_0107 9OL66l59ysIB2YCS 475.0 480.0     1
384371_9OL66l59ysIB2YCS_0108 9OL66l59ysIB2YCS 480.0 487.0     1
384371_9OL66l59ysIB2YCS_0109 9OL66l59ysIB2YCS 487.0 492.0     1
384371_9OL66l59ysIB2YCS_0110 9OL66l59ysIB2YCS 492.0 498.0     1
996368_n4ACC4gnqW0IM1wW_0085 n4ACC4gnqW0IM1wW 629.0 634.02    1
Length: 52824, dtype: int64

In [ ]:
df_segments.rename(columns= {'chunk_IDS':'chunk_ID'}, inplace=True)
df_segments

In [ ]:
top_row = pd.DataFrame({'chunk_ID':['103085_w5Jyq3XMbb3WwiKQ_0000'], 'audio_file_ID':['w5Jyq3XMbb3WwiKQ'], 'start_time':[0.0], 'end_time':[8.0]})
df_segments = pd.concat([top_row, df_segments]).reset_index(drop=True)
df_segments

In [ ]:
df_segments.columns

In [ ]:
df_text.rename(columns={'103085_w5Jyq3XMbb3WwiKQ_0000 लिबर ऑफिस impress में एक प्रस्तुति document बनाना और बुनियादी formatting के इस spoken tutorial में आपका स्वागत है': 'sentence'}, inplace =True)

In [ ]:
df_text

In [ ]:
new = df_text['sentence'].str.split(" ", n= 1, expand = True)
new

In [ ]:
df_text['chunk_ID'] = new[0]
df_text['sentence'] = new[1]

In [ ]:
df_text

In [ ]:
top_row_1 = pd.DataFrame({'chunk_ID':['103085_w5Jyq3XMbb3WwiKQ_0000'], 'sentence':['लिबर ऑफिस impress में एक प्रस्तुति document बनाना और बुनियादी formatting के इस spoken tutorial में आपका स्वागत है']})
df_text = pd.concat([top_row_1, df_text]).reset_index(drop=True)
df_text

In [ ]:
#!pip install pydub

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
from pydub import AudioSegment
import math

class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename
        
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/' + split_filename, format="wav")
        
    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                print('All splited successfully')

In [ ]:
folder = '/content/sample_data/audio'  
file = 'sampleaudio15plus.wav'
split_wav = SplitWavAudioMubin(folder, file)
split_wav.multiple_split(min_per_split=1)